<br><br>
<h1 style="font-size:36px" align="center"> Create & Validate MSA </h1><br><br><br><br><br><br>

In [ ]:
import os
import glob
from Bio import SeqIO
from Bio.Seq import Seq
from tqdm.auto import tqdm
import pylev
import matplotlib.pyplot as plt
import numpy as np
import copy
from Bio import AlignIO

YcaO_HMM_filename = "../raw_sequences/MSA/YcaO_HMM.hmm"

work_dir = "../processed_sequences/initial_MSA"
if not os.path.exists(work_dir):
    os.makedirs(work_dir)
clustalo_output_filename = os.path.join(work_dir,"clustalo_MSA.fa")
mafft_output_filename = os.path.join(work_dir,"mafft_MSA.fa")
hmm_output_filename = os.path.join(work_dir,"hmm_MSA")



<h3 style="font-size:24px">Define algorithm and input sequences</h3><br>

In [ ]:
MSA_input_filename = "../processed_sequences/initial_dataset/alignment_input_sequences.txt"#Don't change me unless you know what you're doing
MSA_algorithm = "hmmalign" #options include mafft, clustalo, hmmalign


custom_alignment_filename = os.path.join(work_dir,"custom")


<h3 style="font-size:24px">Run MSA algorithm</h3><br>

In [ ]:

if (MSA_algorithm == "mafft"):
    !mafft  --distout {MSA_input_filename} > $mafft_output_filename
elif (MSA_algorithm == "clustalo"):
    !clustalo -i {MSA_input_filename} --hmm-in={YcaO_HMM_filename} -o {clustalo_output_filename}
elif (MSA_algorithm == "hmmalign"):
    !hmmalign --trim --amino -o {hmm_output_filename+".sto"} {YcaO_HMM_filename} {MSA_input_filename}
    alignment = AlignIO.read(hmm_output_filename+".sto", "stockholm")
    for i in range(0,len(alignment)):
        alignment[i].seq = Seq((str(alignment[i].seq)).upper())
    AlignIO.write(alignment, hmm_output_filename+".fa", "fasta")
elif (MSA_algorithm == "custom"):
    print()
        

<br><br>
<h1 style="font-size:36px" align="center">Rerun MSA for phylogenetic analysis </h1><br><br><br><br><br><br>

In [2]:
import os
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import AlignIO
YcaO_HMM_filename = "../raw_sequences/MSA/YcaO_HMM.hmm"


def remove_gaps(seq_record):
    seq = seq_record.seq
    seq_without_gaps = Seq("".join(str(seq).split("-")))
    return SeqRecord(seq_without_gaps, id=seq_record.id, description=seq_record.description)

work_dir = "../processed_sequences/hmm_align_no_cluster_050323/"
if not os.path.exists(work_dir):
    os.makedirs(work_dir)

<h3 style="font-size:24px">Define algorithm and input sequences</h3><br>

In [3]:

MSA_input_filename = "../processed_sequences/hmm_align_no_cluster_050323/hmmalign_no_cluster_95_ID_initial_MSA.fa"
MSA_algorithm = "hmmalign" #options include mafft, clustalo, hmmalign
output_MSA_filename = os.path.join(work_dir,"hmmalign_no_cluster_95_ID_final_MSA.fa")


hmm_temp_filename = os.path.join(work_dir,"temp_hmm.fa")

In [4]:
if (MSA_algorithm == "mafft"):
    !mafft  --distout {MSA_input_filename} > $mafft_output_filename
elif (MSA_algorithm == "clustalo"):
    !clustalo -i {MSA_input_filename} --hmm-in={YcaO_HMM_filename} -o {output_MSA_filename}
elif (MSA_algorithm == "hmmalign"):
    output_MSA_filename = output_MSA_filename.split(".fa")[0]
    sequences = [seqrec for seqrec in SeqIO.parse(MSA_input_filename,"fasta")]
    ungapped_sequences = []
    for seq in sequences:
        ungapped_sequences.append(remove_gaps(seq))
    SeqIO.write(ungapped_sequences, hmm_temp_filename, "fasta")
    !hmmalign --trim --amino -o {output_MSA_filename+".sto"} {YcaO_HMM_filename} {hmm_temp_filename}
    alignment = AlignIO.read(output_MSA_filename+".sto", "stockholm")
    for i in range(0,len(alignment)):
        alignment[i].seq = Seq((str(alignment[i].seq)).upper())
    AlignIO.write(alignment, output_MSA_filename+".fa", "fasta")
elif (MSA_algorithm == "custom"):
    print()
else:
    raise Exception("Wrong MSA algorithm selected, options are: mafft, clustalo or hmmalign")

<br><br>
<h1 style="font-size:36px" align="center">Validate MSA </h1><br><br><br><br><br><br>

In [ ]:
import matplotlib.pyplot as plt
import json
from Bio import SeqIO
from tqdm.auto import tqdm
import numpy as np
import os
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import matplotlib as mpl
from matplotlib.patches import Patch
mpl.rcParams['figure.dpi']= 600

helix_data_filename = "../raw_sequences/secondary_structure_for_MSA_validation.json"
helix_data = []
with open(helix_data_filename, "r") as f:
    helix_data = json.load(f)


<h3 style="font-size:24px">Define MSA sequences</h3><br>

In [ ]:
MSA_filename = mafft_output_filename

alignment_sequences = [seqrec for seqrec in SeqIO.parse(MSA_filename,"fasta")]

In [ ]:

# define the starting directory
root_dir = "../raw_sequences"

fasta_paths = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        if  "fasta.fa" in file.lower():
            fasta_path = os.path.join(dirpath, file)
            name_path = os.path.join(dirpath, "name.txt")
            
            # check if the name file exists
            if os.path.isfile(name_path):
                # read the content of the name file
                with open(name_path, "r") as f:
                    name = f.read().strip()
            else:
                raise Exception(f"Name file not found {name_path}")
            # append the fasta path and the name to the list as a tuple
            fasta_paths.append((fasta_path, name))



db_dir = os.path.join(work_dir,"db_data/")
db_filename = os.path.join(db_dir,"my_blast_db")
db_sequences_filename = os.path.join(db_dir,"db_sequences.fa")

if not os.path.exists(db_dir):
    os.mkdir(db_dir)
else:
    !rm -rf $db_dir
    os.mkdir(db_dir)

db_sequences = []
for sequence in alignment_sequences:
    seq = sequence.seq
    seq_without_gaps = Seq("".join(str(seq).split("-")))
    db_sequences.append(SeqRecord(seq_without_gaps, id=sequence.id, description=sequence.description))
SeqIO.write(db_sequences, db_sequences_filename, "fasta")
!makeblastdb -in {db_sequences_filename} -dbtype prot -out {db_filename}

import subprocess

path_accessions = []
for i in tqdm(range(0,len(fasta_paths)),desc="Loading Accession Codes"):
    fasta_path = fasta_paths[i][0]
#     print(f"{i+1}/{len(fasta_paths)} Processing {fasta_path}")
    shortened_path = fasta_paths[i][1]
    output = subprocess.run(f"blastp -db {db_filename} -query {fasta_path} -outfmt '6 sseqid' -max_target_seqs 1 -evalue 1e-50", shell=True, capture_output=True)
    accession_numbers = output.stdout.decode().strip().split("\n")
#     print(f"Got results:\n{accession_numbers}")
    path_accessions.append((shortened_path, accession_numbers))
    
# print(path_accessions)

In [ ]:
def get_item_by_accession(seqs, accession):
    for item in seqs:
        if item['Accession_Interpro'] == accession.split(".")[0]:
            return item
    return None
YcaO_data = []
all_annotations_filename = "../raw_sequences/interpro_all_YcaO_annotated.json"
with open(all_annotations_filename, 'r') as f:
    YcaO_data = json.load(f)

# Create a figure and axis
fig, ax = plt.subplots(figsize=(20, 4))

# Set the y-limits
ax.set_ylim(0, len(helix_data))
negative_xlim = -800
positive_xlim = 450
ax.set_xlim(negative_xlim, len(alignment_sequences[0])+positive_xlim)
ax.set_xticks(range(0, len(alignment_sequences[0]), 200))
processed_secondary_structure_data = []
rect_height = 4/5
for i in tqdm(range(0,len(helix_data))):
    file = helix_data[i]
    accession = file["accession"]
    name = ""
    for path_accession in path_accessions:
        if(path_accession[1][0] == accession):
            name = path_accession[0]
    ax.annotate(name, (negative_xlim, i+rect_height/2),
                                color='black', weight='bold', fontsize=6,
                                ha='left', va='center')
    
    for sequence in alignment_sequences:
        if(sequence.id == accession):
            seq_info = get_item_by_accession(YcaO_data,accession)
            domain_start = seq_info["YcaO_domain"]["start"]+1
            domain_end = seq_info["YcaO_domain"]["end"]+1
            actual_pos = domain_start
            pos_array = []
            for char in sequence.seq:
                if char != "-":
                    actual_pos += 1
                pos_array.append(actual_pos)
            helix_positions = []
            start_helix = 0
            for helix in file["h"]:
                if(helix[1]>domain_start and helix[2]<domain_end):
                    helix_length = helix[2]-helix[1]
                    MSA_length = pos_array.index(helix[2])-pos_array.index(helix[1])
                    stretch_factor = helix_length/MSA_length
                    helix_positions.append([helix[0],pos_array.index(helix[1]),pos_array.index(helix[2])])
                    x_start, x_end = pos_array.index(helix[1]),pos_array.index(helix[2])
                    rect_width = x_end - x_start
                    rect_height = 4/5
                    rect = plt.Rectangle((x_start, i), rect_width, rect_height, alpha=stretch_factor)
                    ax.add_patch(rect)
                    ax.annotate(f'{round(stretch_factor,2)}', (x_start+rect_width/2, i+rect_height/2),
                                color='black', weight='bold', fontsize=8,
                                ha='center', va='center')
                    
                else:
                    start_helix +=1
            sheet_positions = []
            for sheet in file["b"]:
                if(sheet[1]>domain_start and sheet[2]<domain_end):
                    sheet_length = sheet[2]-sheet[1]
                    MSA_length = pos_array.index(sheet[2])-pos_array.index(sheet[1])
                    stretch_factor = sheet_length/MSA_length
                    sheet_positions.append([sheet[0],pos_array.index(sheet[1]),pos_array.index(sheet[2])])
                    x_start, x_end = pos_array.index(sheet[1]),pos_array.index(sheet[2])
                    rect_width = x_end - x_start
                    
                    rect = plt.Rectangle((x_start, i), rect_width, rect_height, alpha=stretch_factor,color="red")
                    ax.add_patch(rect)
                    ax.annotate(f'{round(stretch_factor,2)}', (x_start+rect_width/2, i+rect_height/2),
                                color='black', weight='bold', fontsize=8,
                                ha='center', va='center')
            processed_secondary_structure_data.append({"accession":accession,"h":helix_positions,"b":sheet_positions})

# Set the x-limits and remove the y-axis ticks and labels
ax.set_yticks([])
ax.set_yticklabels([])
ax.set_title('Alignment of secondary structure elements in MAFFT Multiple Sequence Alignment')
ax.set_xlabel('Residue position')
ax.set_ylabel('Protein')
# Show the plot
legend_elements = [    Patch(facecolor='red', alpha=0.5, label='Beta Sheets'),    Patch(facecolor='blue', alpha=0.5, label='Alpha Helices')]
ax.legend(handles=legend_elements, loc='upper right')
plt.show()


In [ ]:
occupancy_data = np.zeros(len(alignment_sequences))
for i in tqdm(range(0,len(alignment_sequences)),desc="getting occupancy data"):
    sequence = alignment_sequences[i]
    for i in range(0,len(sequence.seq)):
        char = sequence.seq[i]
        if(char != "-"):
            occupancy_data[i] += 1

occupancy_data = occupancy_data/len(alignment_sequences)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 2))
ax.set_ylim(0, 1)
ax.set_xlim(negative_xlim, len(alignment_sequences[0])+positive_xlim)
ax.set_xticks(range(0, len(alignment_sequences[0]), 200))
im = ax.imshow(np.array([occupancy_data, occupancy_data]), cmap='gray_r', aspect='auto', vmin=0, vmax=1)
ax.set_yticks([])
ax.set_yticklabels([])
ax.set_title('Occupancy of each position in MAFFT MSA')
ax.set_xlabel('Residue position')
plt.colorbar(im, label='% Occupancy')
plt.show()